In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [4]:
training_data = datasets.FashionMNIST(root="../data/", train=True, download=False, transform=ToTensor())
test_data = datasets.FashionMNIST(root="../data/", train=False, download=False, transform=ToTensor())

In [5]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512), nn.ReLU(),
                                               nn.Linear(512, 512), nn.ReLU(),
                                               nn.Linear(512, 16))
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [7]:
loss_fn = nn.CrossEntropyLoss()

In [10]:
#hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5


In [13]:
# 至少要传入两个参数，一个时需要训练的参数，另一个时学习率
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)


In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {100*correct:>.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
for t in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

loss: 2.775205  [    0/  938]
loss: 2.747039  [ 6400/  938]
loss: 2.709623  [12800/  938]
loss: 2.687299  [19200/  938]
loss: 2.636933  [25600/  938]
loss: 2.578688  [32000/  938]
loss: 2.564315  [38400/  938]
loss: 2.491624  [44800/  938]
loss: 2.463699  [51200/  938]
loss: 2.393263  [57600/  938]
Test Error:
 Accuracy: 27.1%, Avg loss: 2.371620 

loss: 2.379606  [    0/  938]
loss: 2.357821  [ 6400/  938]
loss: 2.251270  [12800/  938]
loss: 2.274526  [19200/  938]
loss: 2.194900  [25600/  938]
loss: 2.078989  [32000/  938]
loss: 2.138169  [38400/  938]
loss: 2.023787  [44800/  938]
loss: 2.036439  [51200/  938]
loss: 1.929286  [57600/  938]
Test Error:
 Accuracy: 56.4%, Avg loss: 1.931481 

loss: 1.965928  [    0/  938]
loss: 1.925365  [ 6400/  938]
loss: 1.787874  [12800/  938]
loss: 1.840065  [19200/  938]
loss: 1.711188  [25600/  938]
loss: 1.649226  [32000/  938]
loss: 1.665696  [38400/  938]
loss: 1.567607  [44800/  938]
loss: 1.588728  [51200/  938]
loss: 1.461478  [57600/  938